# Walmart Retail Sales Forecasting System

## 1. Objective and Dataset
The objective of this project is to develop a comprehensive retail sales forecasting system for Walmart using historical sales data.

## 2. Import Libraries and Define Constants
```python

In [ ]:
# Import functions from src directory
from retailer_sales_forecasting_system.data.extract import load_sales

# Standard library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Set styles for better visualization
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Constants

TEST_SIZE = 13 # last 13 weeks for testing
SEASONALITY_PERIOD = 13 # quarterly seasonality (13 weeks)

In [ ]:
# Load the sales data

df = load_sales()
df.head()

## 3. Exploratory Data Analysis (EDA)

In [ ]:
print("Walmart Retail Sales Data Analysis")
print("==================================")
print("\n📊 Data Overview")
print("==================================")
df.info()
print("\nFeatures: ", df.columns.tolist())

In [ ]:
print("📉 Data Dimensions")
print("=======================================")
print(f"✔️ Number of Records: {df.shape[0]}")
print(f"✔️ Number of Stores: {df['store'].nunique()}")
print(f"✔️ Data Range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"✔️ Total Number of Weeks in Data: {df['date'].nunique()}")
print(f"✔️ Records per Store (Average): {df.shape[0] // df['store'].nunique():.1f}")

In [ ]:
# Confirm if any missing values
missing_values = df.isnull().sum()
print("🔍 Missing Values Check:", missing_values[missing_values > 0] if missing_values.any() else "No missing values found.")

Observations and Insights

## 4. Correlation and Variance Explanation by External Factors

### A. Correlation Analysis - External Factors Impact on Sales

In [ ]:
correlation_features = ['weekly_sales', 'holiday_flag', 'temperature', 'fuel_price', 'cpi', 'unemployment']

# Prepare data for correlation analysis
x = df[correlation_features].copy()

# Map boolean 'holiday_flag' to 0/1
if 'holiday_flag' in x.columns:
    x['holiday_flag'] = x['holiday_flag'].astype('Int64')

# Coerce other features to numeric, ignoring errors
for c in x.columns:
    if c != 'holiday_flag':
        x[c] = pd.to_numeric(x[c], errors='coerce')

correlation_df = x.corr()

print("\n🔗 Correlation of Features with Weekly Sales:")
if 'weekly_sales' in correlation_df:
    correlation_with_sales = correlation_df['weekly_sales'].drop('weekly_sales').sort_values(ascending=False)
    for feature, corr_value in correlation_with_sales.items():
        if feature != 'weekly_sales':
            print(f" ∙ {feature:20s}: {corr_value:+.4f}")
    
else:
    print("No correlation data available for 'weekly_sales'.")

# Heatmap of correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_df, 
            annot=True, 
            cmap='coolwarm', 
            center=0, 
            fmt='.3f',
            square=True,
            linewidths=1,
            cbar_kws={"shrink": .8}
)
plt.title("Correlation Matrix: Sales and External Factors", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

Observations and Insights

### B. Variance Explanation - External Factors Contribution to Sales Variance

**Observations and Insights**

In [ ]:
# Temperature variance explanation (R² calculation)
X_temperature = df[['temperature']].dropna()
y_sales = df.loc[X_temperature.index, 'weekly_sales']

temperature_model = LinearRegression()
temperature_model.fit(X_temperature, y_sales)
r2_temperature = temperature_model.score(X_temperature, y_sales)

print(f"\n🌡️ Variance Explained Analysis:")
print("===================================")
print(f"\nR² Score for Temperature: {r2_temperature:.4f} ({r2_temperature * 100:.2f}% of variance explained by temperature)")

# Multiple Linear Regression for all external factors
# Create a copy to avoid modifying original dataframe
df_copy = df.copy()
# Convert boolean to 0/1 for regression
df_copy['holiday_flag'] = df_copy['holiday_flag'].astype('Int64')

# Prepare data for multiple linear regression
X_multiple_lr = df_copy[['holiday_flag', 'temperature', 'fuel_price', 'cpi', 'unemployment']].dropna()
y_multiple_lr = df_copy.loc[X_multiple_lr.index, 'weekly_sales']

multiple_lr_model = LinearRegression()
multiple_lr_model.fit(X_multiple_lr, y_multiple_lr)
r2_multiple_lr = multiple_lr_model.score(X_multiple_lr, y_multiple_lr)

print(f"R² Score for All Features: {r2_multiple_lr:.4f} ({r2_multiple_lr * 100:.2f}% of variance explained by all factors combined)")

# Feature Importance
feature_importance = pd.DataFrame({
    'Feature': X_multiple_lr.columns,
    'Coefficient': multiple_lr_model.coef_
}).sort_values(by='Feature')

print("\n🔑 Feature Importance based on Coefficients:")
for _, row in feature_importance.iterrows():
    print(f" ∙ {row['Feature']:15s}: {row['Coefficient']:+,.2f}")

## 5. Store-wise Analysis and Sales Distribution

In [ ]:
import math
print("🏬 Store-wise Analysis")
print("======================")

# Total sales per store
store_sales = df.groupby('store')['weekly_sales'].sum().sort_values(ascending=False)

# Print Top 10 Stores by Total Sales
print("Top 10 Stores by Total Sales:")
for rank, (store, sales) in enumerate(store_sales.head(10).items(), start=1):
    print(f"{rank:2d}. Store {store:3d}: ${sales:,.2f}")

# Plot Total Sales per Store of Top 10 Stores
plt.figure(figsize=(12, 6))
top10_sales = store_sales.head(10)
labels = [f"Store {store}" for store in top10_sales.index]
ax = sns.barplot(x=labels, y=top10_sales.values, hue=labels, palette='viridis', legend=False)
plt.title("Total Sales per Store (Top 10)", fontsize=14, fontweight='bold')
# plt.xlabel("Store")
plt.ylabel("Total Weekly Sales ($)")
plt.xticks(rotation=45)

# Format y-axis in $50.0M increments
max_sales = top10_sales.max()
step = 50_000_000
upper = int(math.ceil(max_sales / step) * step)
ax.set_yticks(range(0, upper + step, step))
ax.yaxis.set_major_formatter(lambda x, pos: f"${x/1e6:.1f}M")

# Add labels above bars
for bar in ax.patches:
    height = bar.get_height()
    ax.annotate(
        f"${height/1e6:.1f}M",
        (bar.get_x() + bar.get_width() / 2, height),
        ha='center',
        va='bottom',
        xytext=(0, 4),
        textcoords='offset points'
    )

plt.tight_layout()
plt.show()


In [ ]:
# Weekly Sales Trend for Top 5 Stores
df_indexed = df.set_index('date')
top_5_stores = store_sales.head(5).index

# Plot weekly sales trend for top 5 stores
plt.figure(figsize=(14, 7))
for store in top_5_stores:
    store_data = df_indexed[df_indexed['store'] == store]['weekly_sales'].resample('W').sum()
    plt.plot(store_data.index, store_data.values, label=f'Store {store}', linewidth=2, alpha=0.8)

plt.title("Weekly Sales Trend: Top 5 Performing Stores", fontsize=14, fontweight='bold')
plt.xlabel("Date")
plt.ylabel("Weekly Sales ($)", fontsize=12)
plt.legend(loc="best")
plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${int(x/1e3):.0f}K'))
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**Observations and Insights**

## 6. Event-Based (Holiday) Sales Analysis

In [ ]:
# Separate holiday and non-holiday sales data
holiday_sales = df[df['holiday_flag'] == 1]['weekly_sales']
non_holiday_sales = df[df['holiday_flag'] == 0]['weekly_sales']

# Summarize holiday impact analysis
print("📊 Event-Based (Holiday) Sales Analysis")
print("=======================================")
print(f" ∙ Average Weekly Sales on Holidays   : ${holiday_sales.mean():,.2f}")
print(f" ∙ Average Weekly Sales on Non-Holidays: ${non_holiday_sales.mean():,.2f}")
print(f" ∙ Holiday Impact: {(holiday_sales.mean() / non_holiday_sales.mean() - 1) * 100:+.2f}%")
print(f" ∙ Holiday Weeks in Dataset: {len(df[df['holiday_flag'] == 1]['date'].unique())}")

In [ ]:
# Visualize Holiday vs Non-Holiday Sales distribution

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
axes[0].boxplot([holiday_sales, non_holiday_sales], tick_labels=['Holiday', 'Non-Holiday'])
axes[0].set_title("Sales Distribution: Holiday vs Non-Holiday", fontweight='bold')
axes[0].set_ylabel("Weekly Sales ($)", fontsize=12)
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${int(x/1e3):.0f}K'))
axes[0].grid(axis='y', alpha=0.3)

# Bar plot
comparison_df = pd.DataFrame({
    'Category': ['Holiday', 'Non-Holiday'],
    'Average Sales': [holiday_sales.mean(), non_holiday_sales.mean()]
})

bars = axes[1].bar(comparison_df['Category'], comparison_df['Average Sales'], color=['coral', 'steelblue'], alpha=0.8)
axes[1].set_title("Average Sales Comparison: Holiday vs Non-Holiday", fontweight='bold')
axes[1].set_ylabel("Average Weekly Sales ($)", fontsize=12)
axes[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${int(x/1e3):.0f}K'))

for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width() / 2, height,
                 f'${height:,.0f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

axes[1].grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

**Observations and Insights**

### More Sales Analysis

In [ ]:
# ---- helpers ----
def money_k(x, _):
    # $12K formatting; robust to floats
    return f'${x/1e3:,.0f}K'

money_formatter = mticker.FuncFormatter(money_k)

# Ensure we have Date + Weekly_Sales available and clean
# If Date is index, pull it without full reset_index
if 'date' not in df.columns:
    date = df.index
    weekly_sales = df['weekly_sales']
    df_plot = df.assign(Date=date)
else:
    df_plot = df

# Drop NaNs once
df_plot = df_plot[['date', 'weekly_sales']].dropna()
df_plot = df_plot.assign(
    Year=df_plot['date'].dt.year,
    Quarter=df_plot['date'].dt.quarter
)

sales = df_plot['weekly_sales'].to_numpy()

print("\n📈 1. Sales Distribution Analysis")
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# ---- 1) Histogram (if you truly want KDE, use seaborn or manual density estimate) ----
axes[0, 0].hist(sales, bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].set(
    xlabel='Weekly Sales ($)',
    ylabel='Frequency',
    title='Distribution of Weekly Sales'
)
axes[0, 0].title.set_fontweight('bold')
axes[0, 0].xaxis.set_major_formatter(money_formatter)
axes[0, 0].grid(alpha=0.3)

# ---- 2) Q-Q plot ----
stats.probplot(sales, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Q-Q Plot - Normality Test', fontweight='bold', fontsize=12)
axes[0, 1].grid(alpha=0.3)

# ---- Precompute grouped arrays ONCE (avoids repeated boolean filtering) ----
year_groups = (
    df_plot.groupby('Year', sort=True)['weekly_sales']
    .apply(lambda s: s.to_numpy())
)
years = year_groups.index.to_list()
year_data = year_groups.to_list()

# ---- 3) Boxplot by year ----
# axes[1, 0].boxplot(year_data, tick_labels=years)
# axes[1, 0].set(
#     xlabel='Year',
#     ylabel='Weekly Sales ($)',
#     title='Sales Distribution by Year'
# )
# axes[1, 0].title.set_fontweight('bold')
# axes[1, 0].yaxis.set_major_formatter(money_formatter)
# axes[1, 0].grid(alpha=0.3)

# ---- NEED TO FURTHER REVISE ----
# Box plot still not optimal; try grouping once and reusing

df_copy = df.copy()
if 'date' not in df_copy.columns:
    df_copy = df_copy.reset_index()

# Make sure Date is datetime (safety)
df_copy['date'] = pd.to_datetime(df_copy['date'], errors='coerce')
# Year column
df_copy['year'] = df_copy['date'].dt.year

# Group once instead of filtering repeatedly
year_groups = df_copy.groupby('year', sort=True)['weekly_sales']
years = year_groups.size().index.tolist()
year_data = [s.dropna().to_numpy() for _, s in year_groups]

axes[1, 0].boxplot(
    year_data,
    tick_labels=years,  # Matplotlib 3.9+ rename (was labels=)
    whis=(5, 95),  # Show 5th to 95th percentiles
    showfliers=False  # Hide outliers for clarity
)

axes[1, 0].set_xlabel('Year', fontsize=11)
axes[1, 0].set_ylabel('Weekly Sales ($)', fontsize=11)
axes[1, 0].set_title('Sales Distribution by Year', fontweight='bold', fontsize=12)
axes[1, 0].yaxis.set_major_formatter(
    mticker.FuncFormatter(lambda x, _: f'${int(x/1e3):.0f}K')
)
axes[1, 0].grid(alpha=0.3)

# ---- 4) Violin by quarter (ensure ordering 1..4) ----
q_groups = (
    df_plot.groupby('Quarter', sort=True)['weekly_sales']
    .apply(lambda s: s.to_numpy())
)
quarter_order = [1, 2, 3, 4]
quarter_data = [q_groups.get(q, []) for q in quarter_order]

axes[1, 1].violinplot(
    quarter_data,
    positions=quarter_order,
    showmeans=True,
    showmedians=True
)
axes[1, 1].set(
    xlabel='Quarter',
    ylabel='Weekly Sales ($)',
    title='Sales Distribution by Quarter'
)
axes[1, 1].title.set_fontweight('bold')
axes[1, 1].set_xticks(quarter_order)
axes[1, 1].set_xticklabels(['Q1', 'Q2', 'Q3', 'Q4'])
axes[1, 1].yaxis.set_major_formatter(money_formatter)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
df_copy.groupby('year')['weekly_sales'].agg(['count','nunique','min','median','max'])


In [ ]:
iqr = df_copy.groupby('year')['weekly_sales'].quantile(0.75) - df_copy.groupby('year')['weekly_sales'].quantile(0.25)
iqr
